# Implement data handling of learning data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
get_processed_data_script_file = os.path.join(os.path.pardir,'src','data','get_processed_data.py')

In [39]:
%%writefile $get_processed_data_script_file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def CreateDataFrame(df):
    project_dir = 'C:\\Users\\PLDD\\python\\Python\\ML\\Kaggle\\house_price'
    raw_path = os.path.join(project_dir,'data','raw')
    train_path = os.path.join(raw_path, 'train.csv')
    test_path = os.path.join(raw_path, 'test.csv')
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    test_df['SalePrice'] = 0.0
    df = train_df.append(test_df)
    df.index = list(range(train_df.index.size + test_df.index.size))
################# 2916 out of 2919 Utilities are AllPub so i think it should be droped off
    df = df.drop('Utilities', axis = 1)
############### Handle basment information - replace nan with no_basment or zero quatity
    catbsmflds = ['BsmtQual', 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']
    ambsmflds = ['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','BsmtUnfSF','BsmtFullBath','BsmtHalfBath']
    lx = df[catbsmflds].isnull().all(axis = 1) 
    lx = lx & (df['TotalBsmtSF']==0.0)
    df.loc[lx,catbsmflds] = 'NoBsm'
    df.loc[lx,ambsmflds] = 0.0
############### Handle garage information - replace nan with no_garage or zero quatity    
    catflds = ['GarageType', 'GarageFinish', 'GarageQual','GarageCond']
    yearfld = 'GarageYrBlt'
    amtfld = 'GarageArea'
    lx = (df[catflds].isnull().all(axis = 1)) & (df[amtfld]==0.0)
    df.loc[lx,catflds] = 'NoGarage'
    df.loc[lx,yearfld] = 0
############### Handle Pool information - replace nan with no_pool or zero quatity    
    catfld = 'PoolQC'
    amtfld = 'PoolArea'
    lx = (df[catfld].isnull())& (df[amtfld]==0.0)
    df.loc[lx,catfld] = 'NoPool'
############### Handle FireplaceQu nan - it is only no fireplace #################################
    fld = 'FireplaceQu'
    lx = df['Fireplaces'] == 0
    df.loc[lx,fld] = 'NoFireplace'
############### Handle 'Fence', 'MiscFeature' - just replace it with NoFence or feature###########    
    flds =['Fence', 'MiscFeature']
    df.loc[:,flds] = df[flds].fillna(value = 'Noooo')
    return df
##################################################################################################################
def ResolveNaN(df):
    oldflds = []
    newdf = pd.DataFrame()
######################## 1) MSZoning - 'RL', 'RM', 'C (all)', 'FV', 'RH', nan ################################
    colname = 'MSZoning'
    oldflds.append(colname)
    #### with Mitchel neighbor it is highly likely that zoning is 'RL' (grouping by Neighbors + BuiltYear)
    lx=(df[colname].isnull()) & (df['Neighborhood'] == 'Mitchel')  
    df.loc[lx,colname] = 'RL'
    ### group by IDOTRR neighbor
    invalidlx = df[colname].isnull() 
    idotrrlx = df['Neighborhood'] == 'IDOTRR'

    tempdf = pd.get_dummies(df[colname],prefix = colname).astype(float)
    tempdf.loc[invalidlx,:] = tempdf[idotrrlx].sum().values / tempdf[idotrrlx].sum().sum()
    newdf[tempdf.columns] = tempdf
######################## 2) LotFrontage - float value, nan ################################ 
    #Exclude this field from ML - base on the idea that Lot Area is more important parameter than LotFrontage
    colnames = ['LotFrontage','GarageCars']
    oldflds += colnames
######################## 3) Alley  - nan, 'Grvl', 'Pave' ################################ 
    colname = 'Alley'
    oldflds.append(colname)
    tempdf = pd.get_dummies(df[colname],prefix = colname,dummy_na = True).astype(float)
    newdf[tempdf.columns] = tempdf
######################## 4) 'Exterior1st', 'Exterior2nd','MasVnrType','Electrical','KitchenQual'#############
########################'BsmtQual', 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2'##################
######################## 'Functional','GarageType', 'GarageFinish', 'GarageQual','GarageCond'################
######################## 'PoolQC','SaleType'#################################################################   
    colnames = ['Exterior1st','Exterior2nd','MasVnrType','Electrical','KitchenQual',
                'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
               'Functional','GarageType', 'GarageFinish', 'GarageQual','GarageCond',
               'PoolQC','SaleType']
    for colname in colnames:
        lx = df[colname].isnull()
        oldflds.append(colname)
        tempdf = pd.get_dummies(df[colname],prefix = colname).astype(float)
        tempdf.loc[lx,:] = tempdf[~lx].sum().values / tempdf[~lx].sum().sum()
        newdf[tempdf.columns] = tempdf
####################### 5) Mean apprx - 'MasVnrArea','BsmtFullBath','BsmtHalfBath'###########################
    colnames = ['MasVnrArea','BsmtFullBath','BsmtHalfBath']
    for colname in colnames:
        lx = df[colname].isnull()
        df.loc[lx,colname] = df[~lx][colname].mean()

####################### 6) Median apprx - 'BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','BsmtUnfSF','GarageArea'###
    colnames = ['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','BsmtUnfSF','GarageArea']
    for colname in colnames:
        lx = df[colname].isnull()
        df.loc[lx,colname] = df[~lx][colname].median()
##################### 7) 'GarageYrBlt'####################################################################### 
    colname = 'GarageYrBlt'
    lx = df[colname].isnull()
    df.loc[lx,colname] = df[lx]['YearBuilt']
    
    df = df.drop(labels = oldflds,axis = 1)
    df[newdf.columns] = newdf
################################################Convert int and Objects type fields into float64 ###############
    df = pd.get_dummies(df).astype('float64')
    df.loc[:,'Id'] = df['Id'].astype('int32')
    return df
####################################################################################################################
def FormatAndSave(df):
    dff = df.copy()
    categorical_flds = dff.columns[dff.dtypes == 'category'].values
    dff = pd.get_dummies(dff, columns = categorical_flds)
    
    processed_data_path = os.path.join(os.path.pardir,'data','processed')
    write_train_path = os.path.join(processed_data_path, 'train.csv')
    write_test_path = os.path.join(processed_data_path, 'test.csv')
    
    # train data
    dff.loc[dff.SalePrice != 0].to_csv(write_train_path) 
    # test data
    columns = [column for column in dff.columns if column != 'SalePrice']
    dff.loc[dff.SalePrice == 0, columns].to_csv(write_test_path)
    return df

if __name__ == '__main__':
    df = pd.DataFrame()
    df = df.pipe(CreateDataFrame).pipe(ResolveNaN).pipe(FormatAndSave)

Overwriting ..\src\data\get_processed_data.py


In [40]:
!python $get_processed_data_script_file

In [41]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
write_train_path = os.path.join(processed_data_path, 'train.csv')
write_test_path = os.path.join(processed_data_path, 'test.csv')
train_df = pd.read_csv(write_train_path)

#  Build model

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge

In [43]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_path = os.path.join(processed_data_path, 'train.csv')
test_path = os.path.join(processed_data_path, 'test.csv')
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [51]:
def linear_ml_solve(train_df, test_df, scaler, solver, q = 0.4, target = 'temp.csv'):
    X = train_df.loc[:,train_df.columns != 'SalePrice'].as_matrix().astype('float')
    y = train_df['SalePrice'].ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=q, random_state=0)
    X_train = scaler.fit_transform(X_train)
    if q != 0:
        X_test = scaler.transform(X_test)
        solver.fit(X_train, y_train)
        print(solver.score(X_test, y_test))
    else: 
        print("No test subsets!!!!")
    X = scaler.fit_transform(X)    
    XX = scaler.transform(test_df.as_matrix().astype('float'))
    solver.fit(X, y) 
    pred_y = solver.predict(XX).astype('int')
    df_submission = pd.DataFrame({'Id': test_df['Id'].values, 'SalePrice' : pred_y} )
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path, target)
    df_submission.to_csv(submission_file_path, index=False)

# Preprocessing Scaling

In [52]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

scaler1 = StandardScaler()
scaler2 = MinMaxScaler()
scaler3 = RobustScaler()
scaler4 = Normalizer()
scaler5 = QuantileTransformer()

# ML

In [60]:
np.logspace(-4, 2, 7)


array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [85]:
solver1 = linear_model.RidgeCV(alphas = np.arange(0.001, 5, 0.001))
solver2 = linear_model.LassoCV()
solver3 = linear_model.ElasticNetCV(alphas = np.arange(0.001, 5, 0.001), max_iter = 20000 )
solver4 = linear_model.BayesianRidge()
solver6 = linear_model.PassiveAggressiveRegressor(C = 10,tol = 0.00001, max_iter = 50000)
solver7 = linear_model.HuberRegressor(max_iter=10000)
solver8 = Pipeline([('poly', PolynomialFeatures(degree=2)),  ('linear', solver1)])

param_grid = {
    'alpha': [2e0,1e0,0.5, 0.1, 1e-2, 1e-3,1e-4,1e-5,1e-6],
    'gamma': np.logspace(-4, 2, 14)
}
solver5_1 = GridSearchCV(KernelRidge(kernel = 'polynomial',degree = 80), cv=8, param_grid=param_grid)
solver5_2 = GridSearchCV(KernelRidge(kernel = 'rbf'), cv=8, param_grid=param_grid)

In [86]:

linear_ml_solve(train_df, test_df, scaler5, solver5_2, 0.4,'02_KernelRidge_RBF.csv')

0.8233104015747309


In [ ]:
'01_KernelRidge_polynomial.csv' '01_KernelRidge_RBF.csv'